In [1]:
!pip install vit-keras
!pip install tensorflow_addons
from vit_keras import vit
import tensorflow as tf
from typing import *
from tqdm import tqdm



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=4bc453194753a2b81c60f75b2a0c09e1788927bccd823942f3a5a18fa8eaca46
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 26.3 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
vit_model = vit.vit_b16(
        image_size=224,
        activation='softmax',
        pretrained=True,
        include_top=False,
        pretrained_top=False,
        classes=44)

for layer in vit_model.layers:
    layer.trainable = False


347502902/347502902 [==============================] - 2s 0us/step


/usr/local/lib/python3.10/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


In [4]:
#initializer = tf.keras.initializers.GlorotNormal(seed=CFG_SEED)

In [5]:
def vit_b16_model():    
    vit_b16_sequential = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(224,224,3), dtype=tf.float32, name='input_image'),
        vit_model,
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation='relu'),
        #, kernel_initializer=initializer),
        tf.keras.layers.Dense(256, activation='relu'),
        # kernel_initializer=initializer),
        tf.keras.layers.Dense(44, dtype=tf.float32, activation='softmax'),
        #, kernel_initializer=initializer)
    ], name='vit_b16_sequential_model')
    
    return vit_b16_sequential

In [6]:
# Generate Model
model_vit_b16 = vit_b16_model()

# Generate Summary of the Model
model_vit_b16.summary()

Model: "vit_b16_sequential_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit-b16 (Functional)        (None, 768)               85798656  
                                                                 
 dropout (Dropout)           (None, 768)               0         
                                                                 
 dense (Dense)               (None, 512)               393728    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 44)                11308     
                                                                 
Total params: 86,335,020
Trainable params: 536,364
Non-trainable params: 85,798,656
_________________________________________________________________


In [7]:
CFG_SEED=71
NUM_EPOCHS = 25

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    #directory='training_data/',
    directory='/content/drive/MyDrive/Colab Notebooks/training_data',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))


validation_ds = tf.keras.utils.image_dataset_from_directory(
    #directory='validation_data/',
    directory='/content/drive/MyDrive/Colab Notebooks/validation_data',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))

Found 3581 files belonging to 44 classes.
Found 895 files belonging to 44 classes.


In [9]:
val_batches = tf.data.experimental.cardinality(validation_ds)
print('Number of val batches: %d' % val_batches)
test_dataset = validation_ds.take(val_batches // 5)
validation_data = validation_ds.skip(val_batches // 5)

Number of val batches: 28


In [10]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = validation_data.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
test_ds = test_dataset.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.


In [11]:
# val_batches = tf.data.experimental.cardinality(validation_ds)
# print('Number of val batches: %d' % val_batches)
# test_dataset = validation_ds.take(val_batches // 5)
# validation_data = validation_ds.skip(val_batches // 5)

In [12]:
# Define Early Stopping Callback
earlystopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=3)
callbacks_list = [earlystopping_callback]




In [ ]:
tf.random.set_seed(CFG_SEED)

# Compile the model
model_vit_b16.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    # loss = tf.keras.losses.BinaryCrossentropy(),

    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)


history = model_vit_b16.fit(train_ds,validation_data=val_ds, epochs=NUM_EPOCHS, callbacks=callbacks_list)
print(model_vit_b16.evaluate(test_ds))

Epoch 1/25
112/112 [==============================] - 3784s 33s/step - loss: 2.5959 - accuracy: 0.3083 - val_loss: 1.9170 - val_accuracy: 0.4517
Epoch 2/25
112/112 [==============================] - 3741s 33s/step - loss: 1.6855 - accuracy: 0.5110 - val_loss: 1.6071 - val_accuracy: 0.5279
Epoch 3/25
112/112 [==============================] - 3639s 33s/step - loss: 1.3819 - accuracy: 0.5845 - val_loss: 1.2227 - val_accuracy: 0.6259
Epoch 4/25
112/112 [==============================] - 3633s 32s/step - loss: 1.2015 - accuracy: 0.6336 - val_loss: 1.1652 - val_accuracy: 0.6204
Epoch 5/25
112/112 [==============================] - 3843s 34s/step - loss: 1.0483 - accuracy: 0.6772 - val_loss: 1.0837 - val_accuracy: 0.6721
Epoch 6/25
112/112 [==============================] - 3801s 34s/step - loss: 0.9532 - accuracy: 0.6995 - val_loss: 1.1302 - val_accuracy: 0.6571
Epoch 7/25
112/112 [==============================] - 3859s 34s/step - loss: 0.8864 - accuracy: 0.7177 - val_loss: 1.0939 - val_ac

In [ ]:
# STEP 9: Plot loss and accuracies:
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


In [ ]:
# ## 2 EPOCHS TRAINING: 
#Epoch 1/2
# 112/112 [==============================] - 2820s 25s/step - loss: 2.5552 - accuracy: 0.3250 - val_loss: 1.9654 - val_accuracy: 0.4395
# Epoch 2/2
# 112/112 [==============================] - 2754s 25s/step - loss: 1.6741 - accuracy: 0.5035 - val_loss: 1.5377 - val_accuracy: 0.5401
# 1/5 [=====>........................] - ETA: 1:13 - loss: 1.6646 - accuracy: 0.4375


#5/5 [==============================] - 98s 20s/step - loss: 1.7411 - accuracy: 0.4500
# [1.7410808801651, 0.44999998807907104]